- pip install langchain
- pip install openai langchain-openai
- 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub

chat_llm = ChatOpenAI(
    openai_api_key="#"
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", return_messages=True)

# prompt = PromptTemplate(template="""You are a Neo4j expert having a conversation about how to create Cypher queries

# Chat History: {chat_history}
# Question: {input}
# """, input_variables=["chat_history", "input"])

prompt = PromptTemplate(template="""You are an ontology expert having a conversation about how to create knowledge graphs

Chat History: {chat_history}
Question: {input}
""", input_variables=["chat_history", "input"])

chat_chain = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory)

tools = [
    Tool.from_function(
        name="ChatOpenAI",
        description="Talk about Neo4j Cypher queries",
        func=chat_chain.run,
        return_direct=True,
    )
]

agent_prompt = hub.pull("hwchase17/react")
agent = create_react_agent(chat_llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, handle_parse_errors=True)

while True:
    q = input("> ")
    print(agent_executor.invoke({"input": q}))

>  what is ontology indeed?


{'input': 'what is ontology indeed?', 'chat_history': [HumanMessage(content='Define ontology'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that defines the entities, relationships, and properties within a specific domain. It provides a structured framework for organizing and categorizing information, making it easier to capture, share, and analyze data. By creating an ontology, we can establish a common understanding of the concepts and their interrelationships, which forms the basis for building knowledge graphs.'), HumanMessage(content='what is ontology indeed?'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that defines the entities, relationships, and properties

>  please tell us how to generate an ontology for intermodal transportation


{'input': 'please tell us how to generate an ontology for intermodal transportation', 'chat_history': [HumanMessage(content='Define ontology'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that defines the entities, relationships, and properties within a specific domain. It provides a structured framework for organizing and categorizing information, making it easier to capture, share, and analyze data. By creating an ontology, we can establish a common understanding of the concepts and their interrelationships, which forms the basis for building knowledge graphs.'), HumanMessage(content='what is ontology indeed?'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that def

>  can you give me an example of a knowledge graph in owl?


{'input': 'can you give me an example of a knowledge graph in owl?', 'chat_history': [HumanMessage(content='Define ontology'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that defines the entities, relationships, and properties within a specific domain. It provides a structured framework for organizing and categorizing information, making it easier to capture, share, and analyze data. By creating an ontology, we can establish a common understanding of the concepts and their interrelationships, which forms the basis for building knowledge graphs.'), HumanMessage(content='what is ontology indeed?'), AIMessage(content='Ontology is a branch of philosophy that deals with the nature of being, existence, and reality. In the context of knowledge graphs, ontology refers to a formal representation of knowledge that defines the entities

# Q&A basic

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat_llm = ChatOpenAI(
    openai_api_key="#"
)

prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Question: {question}
""",
    input_variables=["question"],
)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt)

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

{'question': 'What is the weather like?', 'text': "Dude, the weather is totally gnarly today! The waves are pumping and the sun is shining. It's gonna be epic out there! Stoked to catch some sick barrels. How about you, ready to shred some waves?"}


## With context

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat_llm = ChatOpenAI(
    openai_api_key="#"
)
prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Context: {context}
Question: {question}
""",
    input_variables=["context", "question"],
)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like on Watergate Bay?",
    }
)

print(response)

# Providing context is one aspect of Retrieval Augmented Generation (RAG).
# In this program, you manually gave the model context; however, you could have retrieved 
# real-time information from an API or database.


## With Memory

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory

chat_llm = ChatOpenAI(
    openai_api_key="#"
)
prompt = PromptTemplate(
    template="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: {chat_history}
Context: {context}
Question: {question}
""",
    input_variables=["chat_history", "context", "question"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history", input_key="question", return_messages=True
)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory, verbose=True)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while True:
    question = input("> ")
    response = chat_chain.invoke({"context": current_weather, "question": question})

    print(response["text"])

>  I am at Bells. How is the weather there?




> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: []
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: I am at Bells. How is the weather there?


> Finished chain.
Dude, Bells is totally flat and calm right now. No waves to catch, just chill vibes. Why not head over to Fistral instead? They've got sick 6ft waves and offshore winds, perfect for shredding!


>  What about Watergate Bay?




> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: [HumanMessage(content='I am at Bells. How is the weather there?'), AIMessage(content="Dude, Bells is totally flat and calm right now. No waves to catch, just chill vibes. Why not head over to Fistral instead? They've got sick 6ft waves and offshore winds, perfect for shredding!")]
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: What about Watergate Bay?


> Finished chain.
Dude, Watergate Bay has some decent 3ft waves but unfortunately, it's got onshore winds messing up the lineup. Better off heading to Fistral for some sick shredding!


>  Is the wave at Fistral too much?




> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: [HumanMessage(content='I am at Bells. How is the weather there?'), AIMessage(content="Dude, Bells is totally flat and calm right now. No waves to catch, just chill vibes. Why not head over to Fistral instead? They've got sick 6ft waves and offshore winds, perfect for shredding!"), HumanMessage(content='What about Watergate Bay?'), AIMessage(content="Dude, Watergate Bay has some decent 3ft waves but unfortunately, it's got onshore winds messing up the lineup. Better off heading to Fistral for some sick shredding!")]
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: Is 

KeyboardInterrupt: Interrupted by user

# Agents and tools

## Movie trailer agent

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub

llm = ChatOpenAI(
    openai_api_key="#"
)
prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    Chat History:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

while True:
    q = input("> ")
    response = agent_executor.invoke({"input": q})
    print(response["output"])

>  Which one of them has lots of dancing and singing?


I would recommend looking into musical movies, as they often feature lots of dancing and singing. Some popular musicals include "Grease," "The Greatest Showman," "La La Land," "Mamma Mia!," and "Chicago."


>  Which one has Emma Stone in it?


I'm sorry, I'm not sure what you're asking about Emma Stone. Are you looking for movies she has starred in, or do you have a specific question about her?


>  Yes, which movie did she act in?


Emma Stone is a talented actress who has starred in a variety of movies across different genres. Some popular movies that she has appeared in include "La La Land," "The Help," "Easy A," "Birdman," and "The Amazing Spider-Man." If you are specifically looking for a movie with Emma Stone in it, "La La Land" is a great choice as it features her in a leading role and showcases her singing and dancing abilities.


KeyboardInterrupt: Interrupted by user

In [ ]:
```
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True
)
```

## youtube movie


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools import YouTubeSearchTool

llm = ChatOpenAI(
    openai_api_key="#"
)
prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    Chat History:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

youtube = YouTubeSearchTool()

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True,
)

while True:
    q = input("> ")
    response = agent_executor.invoke({"input": q})
    print(response["output"])

>  Find the trailer for the Matrix




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Movie Trailer Search
Action Input: The Matrix['https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUKVGhlIE1hdHJpeA%3D%3D', 'https://www.youtube.com/watch?v=zE7PKRjrid4&pp=ygUKVGhlIE1hdHJpeA%3D%3D']


> Finished chain.
['https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUKVGhlIE1hdHJpeA%3D%3D', 'https://www.youtube.com/watch?v=zE7PKRjrid4&pp=ygUKVGhlIE1hdHJpeA%3D%3D']


>  find a movie about the life of pi




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie Chat
Action Input: movie about the life of piI found two trailers for the movie "Life of Pi":
1. [Life of Pi - Official Trailer #1](https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUKVGhlIE1hdHJpeA%3D%3D)
2. [Life of Pi - Official Trailer #2](https://www.youtube.com/watch?v=zE7PKRjrid4&pp=ygUKVGhlIE1hdHJpeA%3D%3D)


> Finished chain.
I found two trailers for the movie "Life of Pi":
1. [Life of Pi - Official Trailer #1](https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUKVGhlIE1hdHJpeA%3D%3D)
2. [Life of Pi - Official Trailer #2](https://www.youtube.com/watch?v=zE7PKRjrid4&pp=ygUKVGhlIE1hdHJpeA%3D%3D)


>  Find me a movie about the meaning of life




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie Chat
Action Input: Find me a movie about the meaning of lifeI'm sorry, I couldn't find a specific movie about the meaning of life. This is a philosophical concept that may not be the central theme of a particular movie. However, I can recommend movies that touch on existential themes or explore the concept of purpose and meaning in life. Some examples include "The Truman Show," "Eternal Sunshine of the Spotless Mind," "The Fountain," or "The Tree of Life." Let me know if you would like more information on any of these films!


> Finished chain.
I'm sorry, I couldn't find a specific movie about the meaning of life. This is a philosophical concept that may not be the central theme of a particular movie. However, I can recommend movies that touch on existential themes or explore the concept of purpose and meaning in life. Some examples include "The Truman Show," "Eternal Sunshine of the Spotless Mi

>  Find the trailer for the truman show then




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie Trailer Search
Action Input: The Truman Show['https://www.youtube.com/watch?v=dlnmQbPGuls&pp=ygUPVGhlIFRydW1hbiBTaG93', 'https://www.youtube.com/watch?v=MAX9qWiJxvU&pp=ygUPVGhlIFRydW1hbiBTaG93']


> Finished chain.
['https://www.youtube.com/watch?v=dlnmQbPGuls&pp=ygUPVGhlIFRydW1hbiBTaG93', 'https://www.youtube.com/watch?v=MAX9qWiJxvU&pp=ygUPVGhlIFRydW1hbiBTaG93']
